[![Renode](https://dl.antmicro.com/projects/renode/renode.svg)](https://renode.io)

[![Run in Google Colab](https://img.shields.io/badge/-Run%20in%20Google%20colab-%23007ded?logo=google-colab&logoColor=white&style=for-the-badge)](https://colab.research.google.com/github/antmicro/test-colabs/blob/main/boards/thingy53_nrf5340_cpuapp_ns_hello_world.ipynb) [![View ipynb](https://img.shields.io/badge/-View%20ipynb%20source-%23007ded?logo=jupyter&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/thingy53_nrf5340_cpuapp_ns_hello_world.ipynb) [![View Python source](https://img.shields.io/badge/-View%20Python%20source-%23007ded?logo=python&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/thingy53_nrf5340_cpuapp_ns_hello_world.py)

## Install requirements

In [ ]:
! pip install -q git+https://github.com/antmicro/renode-colab-tools.git
! pip install -q git+https://github.com/antmicro/renode-run.git
! pip install -q git+https://github.com/antmicro/pyrenode3.git
! renode-run download --renode-variant dotnet-portable

## Start Renode

In [ ]:
import os
from renode_run import get_default_renode_path
from renode_run.utils import RenodeVariant

os.environ['PYRENODE_RUNTIME'] = 'coreclr'
os.environ['PYRENODE_BIN'] = get_default_renode_path(variant=RenodeVariant.DOTNET_PORTABLE)

from pyrenode3.wrappers import Emulation, Monitor, TerminalTester, LEDTester
from Antmicro.Renode.Peripherals.UART import UARTBackend
from Antmicro.Renode.Analyzers import LoggingUartAnalyzer
from System import String

currentDirectory = os.getcwd()
emulation = Emulation()
monitor = Monitor()
emulation.BackendManager.SetPreferredAnalyzer(UARTBackend, LoggingUartAnalyzer)

## Setup a script

In [ ]:
%%writefile script.resc
logFile $ORIGIN/hello_world-renode.log True

$name?="thingy53_nrf5340_cpuapp_ns"
$bin?=@https://zephyr-dashboard.renode.io/zephyr/bcf4d1ad3856ecb174bc71c4856445fccd093239/thingy53_nrf5340_cpuapp_ns/hello_world/hello_world.elf
$repl?=$ORIGIN/hello_world.repl

using sysbus
mach create $name

machine LoadPlatformDescription @https://zephyr-dashboard.renode.io/zephyr_sim/bcf4d1ad3856ecb174bc71c4856445fccd093239/327f86675b49497a02301a95de5220ccc7bab67d/thingy53_nrf5340_cpuapp_ns/hello_world/hello_world.repl
machine EnableProfiler $ORIGIN/metrics.dump

usbd CreateAndAttachCDCToUARTConverter "boardcdcacmuart"

showAnalyzer boardcdcacmuart

boardcdcacmuart RecordToAsciinema $ORIGIN/hello_world-asciinema
set osPanicHook
"""
self.ErrorLog("OS Panicked")
"""
cpu0 AddSymbolHook "z_fatal_error" $osPanicHook


macro reset
"""
    sysbus LoadELF $bin
    cpu0 VectorTableOffset `sysbus GetSymbolAddress "_vector_table"`
    cpu0 EnableZephyrMode
    cpu0 EnableProfilerCollapsedStack $ORIGIN/hello_world-profile true 62914560 maximumNestedContexts=10
"""

runMacro $reset

## Run the sample

In [ ]:
monitor.execute_script(currentDirectory + "/script.resc")
machine = emulation.get_mach("thingy53_nrf5340_cpuapp_ns")
terminalTester = TerminalTester(machine.sysbus.boardcdcacmuart, 5)

terminalTester.WaitFor(String("Hello World! thingy53/nrf5340/cpuapp/ns"), pauseEmulation=True)

emulation.Dispose()

## UART output

In [ ]:
from renode_colab_tools import asciinema
asciinema.display_asciicast('hello_world-asciinema')

## Renode metrics analysis

In [ ]:
import sys
from pathlib import Path
from renode_run import get_default_renode_path
sys.path.append(str(Path(get_default_renode_path()).parent))

from renode_colab_tools import metrics
from tools.metrics_analyzer.metrics_parser import MetricsParser
metrics.init_notebook_mode(connected=False)
parser = MetricsParser('metrics.dump')

metrics.display_metrics(parser)